In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

2023-01-09 05:59:12,586 - numexpr.utils - INFO
Msg: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.

2023-01-09 05:59:12,586 - numexpr.utils - INFO
Msg: NumExpr defaulting to 8 threads.



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", lang='tr')
            results=[]
            for item in cursor.items(10):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetClear(DataSet)            
 
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            Veri=pd.read_csv('tw.csv')
            Veri["Tür"] = Veri["Tweet_Ayrı"].apply(getTweets.getPolarity)
            Veri["Yoğunluk"] = Veri["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            Veri.to_csv('tw.csv',index=False)
                  
    def TweetTranslate(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet_Degistirilmis"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet["Tweet_Ceviri"][yazi]=Eng_Blob
                DataSet["Tweet_Ceviri"][yazi]=DataSet["Tweet_Ceviri"][yazi].lower()
            print ("Tweetler Çevirildi!")                      
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            getTweets.KelimeAyristirma(DataSet)
            
    def TweetClear(DataSet):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n+|RT')
            for i in range(len(DataSet["Tweet_Degistirilmis"])):        
                DataSet["Tweet_Degistirilmis"][i]=pattern.sub(' ', DataSet["Tweet_Degistirilmis"][i])    
                DataSet["Tweet_Degistirilmis"][i]=DataSet["Tweet_Degistirilmis"][i].lower()              
            DataSet["Tweet_Degistirilmis"]=DataSet["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            getTweets.TweetTranslate(DataSet)
    
    def KelimeAyristirma(DataSet):
        try:
            stop = stopwords.words('english')
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ceviri"].str.split()
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].str.join(" ")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "Kesinlikle"
            elif sub >=0.5 and sub <0.75:
                return "Yüksek İhtimal"
            elif sub >=0.25 and sub <0.50:
                return "Düşük İhtimal"
            else:
                return "Kesin Değil"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [9]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
Tweetler Çevirildi!


In [10]:
Veri=pd.read_csv('tw.csv')
Veri.head()

,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Adı,Tweet_Ayrı,Tür,Yoğunluk
0,@Melike976 merhaba dm gelir misin çok azgınsın,merhaba dm gelir misin çok azgınsın,"hello dm can you come, you are so horny",2023-01-09 03:01:49+00:00,ufuk,merhaba dm gelir misin azgınsın,Nötr,Kesin Değil
1,Güne böyle merhaba diyen herkese GÜNAYDIN 🦁\n\n❤💛🏆 https://t.co/9FBSZeilYa,güne böyle merhaba diyen herkese günaydin,good morning to everyone who says hello to the day,2023-01-09 03:01:31+00:00,𝕬𝖞𝖌𝖚𝖓 𝕬𝖞𝖘𝖊,güne böyle merhaba diyen herkese günaydin,Nötr,Kesin Değil
2,Merhaba @CDZGAMING_ neden #DoNotArgue Clip Creator'ı denemiyorsunuz?\n\nhttps://t.co/oL0mbjRpaI\n\nKolayca #TwitchClips… https://t.co/iyI7iOTy6j,merhaba neden clip creator ı denemiyorsunuz kolayca,hello why don't you try clip creator easily,2023-01-09 03:01:30+00:00,Do Not Argue - no arguments needed,merhaba clip creator ı denemiyorsunuz kolayca,Nötr,Kesin Değil
3,merhaba beni deli sikti manitayla sabahladim simdi de bu saatte kütüphaneye gidiyom 0 uyku ama ben makineyim,merhaba beni deli sikti manitayla sabahladim simdi de bu saatte kütüphaneye gidiyom uyku ama ben makineyim,"hello fucked me crazy manita in the morning, now i go to the library at this hour, but i'm a machine",2023-01-09 03:00:48+00:00,rio👺,merhaba beni deli sikti manitayla sabahladim simdi saatte kütüphaneye gidiyom uyku ben makineyim,Nötr,Kesin Değil
4,@ElifMuratBi1 Merhaba iyi eğlenceler,merhaba iyi eğlenceler,hello good fun,2023-01-09 03:00:24+00:00,yakup,merhaba iyi eğlenceler,Nötr,Kesin Değil


In [11]:
Veri.tail()

,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Adı,Tweet_Ayrı,Tür,Yoğunluk
5,@ifa72821294 Merhaba canım hayırlı sabahlar diliyorum,merhaba canım hayırlı sabahlar diliyorum,"hello dear, i wish you good morning",2023-01-09 02:59:51+00:00,Hüseyin Sert,merhaba canım hayırlı sabahlar diliyorum,Nötr,Kesin Değil
6,@elifturkporno Merhaba ben İstanbul dan olgunum tanışmak isteyen takip dm lütfen,merhaba ben istanbul dan olgunum tanışmak isteyen takip dm lütfen,"hello, i am mature from istanbul.",2023-01-09 02:59:18+00:00,can dağlı,merhaba ben istanbul dan olgunum tanışmak isteyen takip dm lütfen,Nötr,Kesin Değil
7,uyandım...yeni 1 güne merhaba demek için hiç normal 1 saat değil???giyinelim bakalım💅,uyandım yeni güne merhaba demek için hiç normal saat değil giyinelim bakalım,"i woke up to the new day to say hello not a normal time, let's get dressed.",2023-01-09 02:58:53+00:00,ml,uyandım yeni güne merhaba demek normal saat değil giyinelim bakalım,Pozitif,Yüksek İhtimal
8,@ad028m \nMerhaba ben İstanbul Sefaköy den yazıyorum yaşım 25 a.p konuşmak isterim,merhaba ben istanbul sefaköy den yazıyorum yaşım a p konuşmak isterim,"hello, i am writing from istanbul sefaköy i would like to talk to my age",2023-01-09 02:58:43+00:00,Serhat Yıldız,merhaba ben istanbul sefaköy den yazıyorum yaşım a p konuşmak isterim,Nötr,Kesin Değil
9,"RT @konusmasanati: Dostlar başımız sağ olsun!\nÇok üzgünüm\nAsker, polis, öğretmen, doktor, mühendis liyakâtli canlar yitiyor!\nYeter!\n""Sırrı…",dostlar başımız sağ olsun çok üzgünüm asker polis öğretmen doktor mühendis liyakâtli canlar yitiyor yeter sırrı,"friends, thanks to our heads, i am very sorry, soldier police teacher doctor engineer engineer.",2023-01-09 02:57:26+00:00,Angela,dostlar başımız sağ olsun üzgünüm asker polis öğretmen doktor mühendis liyakâtli canlar yitiyor yeter sırrı,Nötr,Kesin Değil


In [12]:
logger = logging.getLogger(__name__)
examples = pd.read_csv('tw.csv')

morphology = TurkishMorphology.create_with_defaults()

# SENTENCE NORMALIZATION
normalizer = TurkishSentenceNormalizer(morphology)

start = time.time()
for example in examples["Tweet_Degistirilmis"]:
    print(example)
    print(normalizer.normalize(example), "\n")
    
sc = TurkishSpellChecker(morphology)


2023-01-09 06:02:22,352 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 5.889094352722168

  merhaba dm gelir misin çok azgınsın
merhaba dm gelir misin çok azgınsın 

güne böyle merhaba diyen herkese günaydin    
güne böyle merhaba diyen herkese günaydın 

merhaba   neden   clip creator ı denemiyorsunuz    kolayca     
merhaba neden klip kreatör ı denemiyorsunuz kolayca 

merhaba beni deli sikti manitayla sabahladim simdi de bu saatte kütüphaneye gidiyom   uyku ama ben makineyim
merhaba beni deli sıktı manitayla sabahladım şimdi de bu saatte kütüphaneye gidiyorum uyku ama ben makineyim 

  merhaba iyi eğlenceler
merhaba iyi eğlenceler 

  merhaba canım hayırlı sabahlar diliyorum
merhaba canım hayırlı sabahlar diliyorum 

  merhaba ben istanbul dan olgunum tanışmak isteyen takip dm lütfen
merhaba ben istanbul dan olgunum tanışmak isteyen takip dm lütfen 

uyandım yeni   güne merhaba demek için hiç normal   saat değil giyinelim bakalım
uyand